In [1]:
import pandas as pd
import os
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import re
from scipy.spatial.distance import cdist


In [ ]:
simulated = mat_data = scipy.io.loadmat('Simulated.mat')
V = simulated['V'].squeeze()
I = simulated['I'].squeeze()
t = simulated['t'].squeeze()


In [2]:
def check(s):
    pattern = r'\d+\.csv$'
    return bool(re.search(pattern, s))

file_list = sorted([f for f in os.listdir(os.getcwd()) if check(f)])

In [ ]:
mod = 0
dfs = []

for i,file_name in enumerate(file_list):
    df = pd.read_csv(file_name,header= None)
    if i==1 or i==2:
      df = df.iloc[::-1]
    if (i == 2 or i ==3) and mod == 1:
      df[1]*=-1

    dfs.append(df)


df = pd.concat(dfs,axis=0, ignore_index=True)

In [ ]:
x = round(t.shape[0]/4)
Segmented_V = []
Segmented_I = []
Segmented_t = []
for i in range(4):
  Segmented_V.append(V[i*x: (i+1)*x + 1])
  Segmented_I.append(I[i*x: (i+1)*x + 1])
  Segmented_t.append(t[i*x: (i+1)*x + 1])


In [ ]:

def find_closest_points(V_exp, V, I):
    distances = cdist(V_exp.reshape(-1, 1), V.reshape(-1, 1))
    closest_indices = np.argmin(distances, axis=1)
    V = V[closest_indices]
    I = I[closest_indices]
    return V, I

In [ ]:
for i in range(4):
  Segmented_V[i], Segmented_I[i] = find_closest_points(dfs[i][0].to_numpy(),np.array(Segmented_V[i]), np.array(Segmented_I[i]))

V = np.array([item for inner_list in Segmented_V for item in inner_list])
I = np.array([item for inner_list in Segmented_I for item in inner_list])
V_exp = df[0].to_numpy()
I_exp = df[1].to_numpy()


In [ ]:
N = V.shape[0]
V_norm = np.sqrt((V_exp**2).sum())
I_norm = np.sqrt((I_exp**2).sum())
v_e = ((V - V_exp)**2).sum()/V_norm
i_e = ((I - I_exp)**2).sum()/I_norm
e = np.sqrt((1/N) *(v_e + i_e))